# Azure AI Foundry

<center><img src="../../../images/Azure-AI-Foundry_1600x900.jpg" alt="Azure AI Foundry" width="600">

## Laboratory 2

In this laboratory we will explore the AI services present in Azure Foundry. This laboratory will cover the following services:
- Speech
- Language + Translator
- Vision + Document 
- Content Safety

Understanding these services allows us to add more capabilities to our applications.

### Exercise 1 - Speech

The Speech service provides speech-to-text and text-to-speech conversion capabilities with a Speech resource. You can transcribe speech to text with high accuracy, produce natural-sounding voices from text to speech, translate spoken audio, and use speaker recognition during conversations. Create custom voices, add specific words to your base vocabulary, or build your own models. Run Speech anywhere, in the cloud or at the edge in containers. It's easy to enable speech in your applications, tools, and devices with the Speech CLI, Speech SDK, and REST APIs.

Common scenarios for using speech:

**Caption Generation:** Learn how to synchronize captions with your input audio, apply profanity filters, get partial results, apply customizations, and identify spoken languages for multilingual scenarios.

**Audio Content Creation:** You can use neural voices to make interactions with chatbots and voice assistants more natural and engaging, convert digital texts like e-books into audiobooks, and enhance automotive navigation systems.

**Call Center:** Transcribe calls in real-time or process a batch of calls, remove personally identifiable information, and extract insights like sentiment analysis to assist with your call center use case.

**Language Learning:** Provide pronunciation assessment feedback for language learners, offer real-time transcription support for remote learning conversations, and read instructional materials aloud using neural voices.

**Voice Assistants:** Create natural, human-like conversational interfaces for your applications and experiences. The voice assistant feature offers fast and reliable interaction between a device and an assistant implementation.

To perform this exercise, make sure your `.env` file has the following variables filled:
- SPEECH_ENDPOINT 
- SPEECH_KEY

After verifying, let's start by loading the necessary libraries, initializing the client, and making a call to convert audio to text:

In [ ]:
%pip install azure-cognitiveservices-speech

In [ ]:
# Continuous speech recognition to process all audio, even with initial silence
import os
import azure.cognitiveservices.speech as speechsdk
import time
from dotenv import load_dotenv

load_dotenv() 

speech_key = os.getenv('SPEECH_KEY')
speech_region = os.getenv('SPEECH_REGION')
audio_file = '../../../samples/audio001.wav'
if speech_key and speech_region:
    try:
        speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
        speech_config.speech_recognition_language = "en-US"
        audio_config = speechsdk.audio.AudioConfig(filename=audio_file)
        speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

        recognized_texts = []
        def recognized_cb(evt):
            if evt.result.text:
                print('Recognized:', evt.result.text)
                recognized_texts.append(evt.result.text)

        speech_recognizer.recognized.connect(recognized_cb)

        print("Starting continuous recognition...")
        speech_recognizer.start_continuous_recognition()

        # Wait for recognition to finish (adjust time according to audio size)
        time.sleep(10)
        speech_recognizer.stop_continuous_recognition()

        print("Recognition finished. Complete text:")
        print(' '.join(recognized_texts))
    except Exception as e:
        print(f"Error in continuous recognition: {e}")
else:
    print("Please configure the SPEECH_KEY and SPEECH_REGION environment variables.")

### Exercise 2 - Language + Translator

Integrate natural language into applications, bots, and IoT devices. For example, this service can remove sensitive data, segment long meetings into chapters, analyze health records, and orchestrate conversational bots with your custom intents using factual responses.

This Language service unifies the following Azure AI services previously available separately: Text Analytics, QnA Maker, and LUIS.

Azure AI Foundry allows you to use most of the following service features without needing to write code.

**Named Entity Recognition (NER)** - Named entity recognition identifies different entries in text and categorizes them into predefined types.

**Personal and health information detection** - PII detection identifies entities in text and conversations (chat or transcriptions) that are associated with individuals.

**Language detection** - Language detection evaluates text and detects a wide range of languages and variant dialects.

**Sentiment analysis and opinion mining** - Sentiment analysis and opinion mining are pre-configured features that help you understand public perception of your brand or topic. These features analyze text to identify positive or negative sentiment and can link them to specific elements within the text.

**Summarization** - Summarization condenses information for text and conversations (chat and transcriptions). Text summarization generates a summary, supporting two approaches: Extractive summarization creates a summary by selecting key phrases from the document and preserving their original positions. In contrast, abstractive summarization generates a summary by producing new, concise, and coherent sentences or phrases that are not directly copied from the original document. Conversation summarization recaps and segments long meetings into chapters with timestamps. Call center summarization summarizes customer problems and their resolutions.

**Key phrase extraction** - Key phrase extraction is a pre-configured feature that evaluates and returns key concepts in unstructured text, returning them as a list.

**Entity linking** - Entity linking is a pre-configured feature that disambiguates the identity of entities (words or phrases) found in unstructured text and returns links to Wikipedia.

**Text analytics for health** - Text analytics for health extracts and labels relevant health information from unstructured text.

**Custom text classification** - Custom text classification allows you to build custom AI models to classify unstructured text documents into custom classes you define.

**Custom Named Entity Recognition (Custom NER)** - Custom NER allows you to build custom AI models to extract custom entity categories (labels for words or phrases), using unstructured text you provide.

**Conversational language understanding** - Conversational language understanding (CLU) allows users to build custom natural language understanding models to predict the overall intent of an incoming utterance and extract important information from it.

**Orchestration workflow** - Orchestration workflow is a custom feature that allows you to connect Conversational Language Understanding (CLU), question answering, and LUIS applications.

**Question answering** - Question answering is a custom feature that identifies the most appropriate answer for user inputs. This feature is typically used to develop conversational client applications, including social media platforms, chat bots, and voice-enabled desktop applications.

In [ ]:
%pip install azure.ai.textanalytics

In [ ]:
# Example - Language Service
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Configure Language client
language_endpoint = os.getenv('AZURE_LANGUAGE_ENDPOINT')
language_key = os.getenv('AZURE_LANGUAGE_KEY')

if language_endpoint and language_key:
    # Create client
    text_analytics_client = TextAnalyticsClient(
        endpoint=language_endpoint,
        credential=AzureKeyCredential(language_key)
    )
    
    # Sample text
    documents = [
        "I love this product! It's amazing and works perfectly.",
        "This service is terrible, it didn't work as expected.",
        "The service was adequate, nothing exceptional."
    ]
    
    try:
        # Sentiment analysis
        response = text_analytics_client.analyze_sentiment(documents=documents, language="en")
        
        print("=== Sentiment Analysis ===")
        for idx, doc in enumerate(response):
            if not doc.is_error:
                print(f"Document {idx + 1}:")
                print(f"  Text: {documents[idx]}")
                print(f"  Sentiment: {doc.sentiment}")
                print(f"  Confidence: Positive={doc.confidence_scores.positive:.2f}, "
                      f"Neutral={doc.confidence_scores.neutral:.2f}, "
                      f"Negative={doc.confidence_scores.negative:.2f}")
                print()
            else:
                print(f"Error in document {idx + 1}: {doc.error}")
                
        # Key phrase extraction
        key_phrases_response = text_analytics_client.extract_key_phrases(documents=documents, language="en")
        
        print("=== Key Phrase Extraction ===")
        for idx, doc in enumerate(key_phrases_response):
            if not doc.is_error:
                print(f"Document {idx + 1}:")
                print(f"  Key phrases: {', '.join(doc.key_phrases)}")
                print()
            else:
                print(f"Error in document {idx + 1}: {doc.error}")
                
    except Exception as e:
        print(f"Error: {e}")
else:
    print("Define the LANGUAGE_ENDPOINT and LANGUAGE_KEY variables in the .env file")

### Exercise 3 - Vision + Document

Give your applications the ability to read text, analyze images, process documents, and detect faces with technologies like optical character recognition (OCR) and machine learning.

The Azure AI Vision service provides access to advanced algorithms that process images and return information based on the visual features you're interested in. The following table lists the main product categories.

**Optical Character Recognition (OCR)** - The Optical Character Recognition (OCR) service extracts text from images. You can use the Read API to extract printed and handwritten text from photos and documents. It uses deep learning-based models and works with text on various surfaces and backgrounds. This includes business documents, invoices, receipts, posters, business cards, letters, and whiteboards. The OCR APIs support extracting printed text in several languages.

**Image Analysis** - The Image Analysis service extracts many visual features from images, such as objects, faces, adult content, and automatically generated text descriptions.

**Face** - The Face service provides AI algorithms that detect, recognize, and analyze human faces in images. Facial recognition software is important in many different scenarios, such as identification, touchless access control, and face blurring for privacy.

**Video Retrieval** - Video Retrieval allows you to create a video index that you can search with natural language.

In [ ]:
import os
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential

# Configure Vision client
vision_endpoint = os.getenv('AZURE_VISION_ENDPOINT')
vision_key = os.getenv('AZURE_VISION_KEY')

if vision_endpoint and vision_key:
    try:
        print("=== Azure AI Vision - Image Analysis ===")
        
        # Create client
        client = ImageAnalysisClient(
            endpoint=vision_endpoint,
            credential=AzureKeyCredential(vision_key)
        )
        
        # Analyze remote image
        print("\n1. Analyzing remote image...")
        result = client.analyze_from_url(
            image_url="https://learn.microsoft.com/azure/ai-services/computer-vision/media/quickstarts/presentation.png",
            visual_features=[VisualFeatures.CAPTION, VisualFeatures.READ],
            gender_neutral_caption=True,  # Optional (default is False)
        )
        
        print("Analysis results:")
        
        # Display caption results
        print("\n📝 Caption:")
        if result.caption is not None:
            print(f"   '{result.caption.text}', Confidence: {result.caption.confidence:.4f}")
        else:
            print("   No caption found")

        # Display OCR results (extracted text)
        print("\n📖 Extracted text (OCR):")
        if result.read is not None:
            for block in result.read.blocks:
                for line in block.lines:
                    print(f"   Line: '{line.text}', Bounding box: {line.bounding_polygon}")
                    for word in line.words:
                        print(f"     Word: '{word.text}', Confidence: {word.confidence:.4f}")
        else:
            print("   No text found")
            
    except Exception as e:
        print(f"Error in image analysis: {e}")
        
else:
    print("Define the AZURE_VISION_ENDPOINT and AZURE_VISION_KEY variables in the .env file")

In [ ]:
# Advanced example - Complete local image analysis
if vision_endpoint and vision_key:
    try:
        print("\n=== Advanced Image Analysis ===")
        
        # Analyze local image
        image_path = "../../../samples/car-accident.png"
        
        print(f"\n2. Analyzing local image: {image_path}")
        
        # Open local image
        with open(image_path, "rb") as image_data:
            # Complete analysis with multiple functionalities
            result = client.analyze(
                image_data=image_data.read(),
                visual_features=[
                    VisualFeatures.CAPTION,
                    VisualFeatures.READ,
                    VisualFeatures.TAGS,
                    VisualFeatures.OBJECTS,
                    VisualFeatures.PEOPLE,
                    VisualFeatures.SMART_CROPS
                ],
                gender_neutral_caption=True
            )
            
            print("\n📝 Caption:")
            if result.caption:
                print(f"   '{result.caption.text}', Confidence: {result.caption.confidence:.4f}")
            
            print("\n🏷️  Identified tags:")
            if result.tags:
                for tag in result.tags.list:
                    print(f"   - {tag.name}: {tag.confidence:.4f}")
            
            print("\n📦 Detected objects:")
            if result.objects:
                for obj in result.objects.list:
                    print(f"   - {obj.tags[0].name}: {obj.tags[0].confidence:.4f}")
                    print(f"     Location: {obj.bounding_box}")
            
            print("\n👥 Detected people:")
            if result.people:
                for person in result.people.list:
                    print(f"   - Person detected with confidence: {person.confidence:.4f}")
                    print(f"     Location: {person.bounding_box}")
            
            print("\n✂️  Smart crops:")
            if result.smart_crops:
                for crop in result.smart_crops.list:
                    print(f"   - Aspect {crop.aspect_ratio}: {crop.bounding_box}")
                    
    except FileNotFoundError:
        print(f"Image file not found: {image_path}")
        print("Make sure the file exists at the specified path.")
    except Exception as e:
        print(f"Error in advanced analysis: {e}")

In [ ]:
# Example - Analysis of different types of images
if vision_endpoint and vision_key:
    try:
        print("\n=== Comparison of Different Image Analyses ===")
        
        # List of images to analyze
        images_to_analyze = [
            {
                "url": "https://learn.microsoft.com/azure/ai-services/computer-vision/media/quickstarts/presentation.png",
                "description": "Calendar presentation"
            },
            {
                "url": "https://images.unsplash.com/photo-1518791841217-8f162f1e1131?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&w=1000&q=80",
                "description": "Cat"
            },
            {
                "url": "https://images.unsplash.com/photo-1506905925346-21bda4d32df4?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&w=1000&q=80",
                "description": "Mountain landscape"
            }
        ]
        
        for i, image_info in enumerate(images_to_analyze, 1):
            print(f"\n{i}. Analyzing: {image_info['description']}")
            print(f"   URL: {image_info['url']}")
            
            try:
                # Analysis focused on caption and tags
                result = client.analyze_from_url(
                    image_url=image_info["url"],
                    visual_features=[VisualFeatures.CAPTION, VisualFeatures.TAGS],
                    gender_neutral_caption=True
                )
                
                if result.caption:
                    print(f"   📝 Caption: '{result.caption.text}' (Confidence: {result.caption.confidence:.4f})")
                
                if result.tags:
                    print("   🏷️  Top 5 tags:")
                    for tag in result.tags.list[:5]:
                        print(f"      - {tag.name}: {tag.confidence:.4f}")
                        
            except Exception as e:
                print(f"   ❌ Error analyzing image {i}: {e}")
                
    except Exception as e:
        print(f"General error in comparative analysis: {e}")

#### Azure AI Vision Features

Azure AI Vision offers various functionalities through the `azure-ai-vision-imageanalysis` SDK:

**📝 Main Features:**
- **CAPTION**: Generates descriptive captions for images
- **READ**: Extracts text from images (OCR)
- **TAGS**: Identifies objects, concepts, and actions in the image
- **OBJECTS**: Detects and locates specific objects
- **PEOPLE**: Identifies people in the image
- **SMART_CROPS**: Suggests intelligent crops of the image
- **FACES**: Detects and analyzes faces (separate functionality)

**📊 Analysis Methods:**
- `analyze_from_url()`: Analyzes image from a URL
- `analyze()`: Analyzes image from binary data (local file)

**⚙️ Important Parameters:**
- `visual_features`: List of features to be analyzed
- `gender_neutral_caption`: Generates gender-neutral captions
- `language`: Language for results (default: English)
- `smart_crops_aspect_ratios`: Aspect ratios for smart crops

In [ ]:
# 🎯 Practical Exercise - Test your own images
if vision_endpoint and vision_key:
    print("=== Practical Exercise ===")
    print("\n📝 Instructions:")
    print("1. Replace the URL below with an image of your choice")
    print("2. Choose the features you want to test")
    print("3. Run the cell and analyze the results")
    
    # 🎯 MODIFY HERE: Paste an image URL to test
    your_image_url = "https://images.unsplash.com/photo-1544947950-fa07a98d237f?ixlib=rb-4.0.3&auto=format&fit=crop&w=800&q=80"
    
    # 🎯 MODIFY HERE: Choose the features you want to test
    selected_features = [
        VisualFeatures.CAPTION,
        VisualFeatures.TAGS,
        VisualFeatures.OBJECTS,
        # VisualFeatures.READ,          # Uncomment for OCR
        # VisualFeatures.PEOPLE,        # Uncomment to detect people
        # VisualFeatures.SMART_CROPS,   # Uncomment for smart crops
    ]
    
    try:
        print(f"\n🔍 Analyzing your image...")
        print(f"URL: {your_image_url}")
        
        result = client.analyze_from_url(
            image_url=your_image_url,
            visual_features=selected_features,
            gender_neutral_caption=True
        )
        
        # Results
        print("\n" + "="*50)
        print("🎉 YOUR ANALYSIS RESULTS")
        print("="*50)
        
        if result.caption:
            print(f"\n📝 CAPTION:")
            print(f"   '{result.caption.text}'")
            print(f"   Confidence: {result.caption.confidence:.2%}")
        
        if result.tags:
            print(f"\n🏷️  IDENTIFIED TAGS:")
            for i, tag in enumerate(result.tags.list[:10], 1):
                print(f"   {i:2d}. {tag.name:<20} {tag.confidence:.2%}")
        
        if result.objects:
            print(f"\n📦 DETECTED OBJECTS:")
            for i, obj in enumerate(result.objects.list, 1):
                print(f"   {i}. {obj.tags[0].name} (Confidence: {obj.tags[0].confidence:.2%})")
        
        if result.read:
            print(f"\n📖 EXTRACTED TEXT:")
            for block in result.read.blocks:
                for line in block.lines:
                    print(f"   '{line.text}'")
        
        if result.people:
            print(f"\n👥 DETECTED PEOPLE: {len(result.people.list)} person(s)")
        
        if result.smart_crops:
            print(f"\n✂️  SUGGESTED CROPS: {len(result.smart_crops.list)} option(s)")
            
        print("\n" + "="*50)
        
    except Exception as e:
        print(f"\n❌ Error analyzing your image: {e}")
        print("Check if the image URL is correct and accessible.")
        
else:
    print("❌ Configure the AZURE_VISION_ENDPOINT and AZURE_VISION_KEY environment variables")

#### 🎓 Next Steps

**📚 Documentation and Resources:**
- [Official Azure AI Vision Documentation](https://learn.microsoft.com/azure/ai-services/computer-vision/)
- [Complete Guide to Image Analysis API 4.0](https://learn.microsoft.com/azure/ai-services/computer-vision/how-to/call-analyze-image-40)
- [Python SDK Reference](https://aka.ms/azsdk/image-analysis/ref-docs/python)
- [Code Samples on GitHub](https://aka.ms/azsdk/image-analysis/samples/python)

**🛠️ Also Try:**
- [Azure AI Vision Studio](https://portal.vision.cognitive.azure.com/) - Visual interface to test features
- [Azure AI Face Service](https://azure.microsoft.com/services/cognitive-services/face/) - Advanced facial analysis
- [Azure AI Custom Vision](https://www.customvision.ai/) - Custom model training
- [Azure AI Video Indexer](https://www.videoindexer.ai/) - Video analysis

**💻 Suggested Practical Projects:**
1. **Image Descriptor for Accessibility**: Create a website that generates automatic alt-text
2. **Document Analyzer**: Extract text from invoices and documents
3. **Content Moderator**: System to automatically classify images
4. **Visual Assistant**: Mobile app that describes the surrounding environment

**🔄 Integration with Other Services:**
- Combine with Azure AI Language for sentiment analysis of extracted text
- Use with Azure AI Translator for automatic translation of text in images
- Integrate with Azure Storage for batch processing
- Connect with Power BI for visual insights dashboards

In [ ]:
# Example - Document Intelligence
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

# Configure Document Intelligence client
doc_intelligence_endpoint = os.getenv('DOC_INTELLIGENCE_ENDPOINT')
doc_intelligence_key = os.getenv('DOC_INTELLIGENCE_KEY')

if doc_intelligence_endpoint and doc_intelligence_key:
    try:
        print("\n=== Document Intelligence ===")
        
        # Create client
        document_analysis_client = DocumentAnalysisClient(
            endpoint=doc_intelligence_endpoint,
            credential=AzureKeyCredential(doc_intelligence_key)
        )
        
        # Analyze generic document
        doc_path = "../../../samples/placa.jpg"
        
        with open(doc_path, "rb") as f:
            # Use pre-built model for general layout
            poller = document_analysis_client.begin_analyze_document(
                "prebuilt-layout", document=f
            )
            
        result = poller.result()
        
        print("Document Layout Analysis:")
        print(f"Number of pages: {len(result.pages)}")
        
        # Extract tables
        if result.tables:
            print(f"\nTables found: {len(result.tables)}")
            for idx, table in enumerate(result.tables):
                print(f"Table {idx + 1}: {table.row_count} rows x {table.column_count} columns")
                
        # Extract paragraphs
        if result.paragraphs:
            print(f"\nParagraphs found: {len(result.paragraphs)}")
            for idx, paragraph in enumerate(result.paragraphs[:3]):  # Show only first 3
                print(f"Paragraph {idx + 1}: {paragraph.content[:100]}...")
                
        # Extract key-value pairs
        if result.key_value_pairs:
            print(f"\nKey-value pairs found: {len(result.key_value_pairs)}")
            for kv_pair in result.key_value_pairs[:5]:  # Show only first 5
                if kv_pair.key and kv_pair.value:
                    print(f"  {kv_pair.key.content}: {kv_pair.value.content}")
                    
    except FileNotFoundError:
        print(f"Document file not found: {doc_path}")
    except Exception as e:
        print(f"Error in Document Intelligence: {e}")
        
else:
    print("Define the DOC_INTELLIGENCE_ENDPOINT and DOC_INTELLIGENCE_KEY variables in the .env file")

### Exercise 4 - Content Safety

Azure AI Content Safety detects harmful user-generated and AI-generated content in applications and services. This service provides various types of analysis.

**Prompt Shields** - Examines text for risks of user input attacks on a Large Language Model.

**Groundedness detection (preview)** - Detects whether text responses from large language models (LLMs) are grounded in the source materials provided by users.

**Protected material detection in text** - Examines AI-generated text for known text content (for example, song lyrics, articles, recipes, selected web content).

**Custom categories API (standard) (preview)** - Allows you to create and train your own custom content categories and examine text for matches.

**Custom categories API (rapid) (preview)** - Allows you to define emerging patterns of harmful content and examine text and images for matches.

**Text analysis API** - Examines text for sexual, violence, hate, and self-harm content with multiple severity levels.

**Image analysis API** - Examines images for sexual, violence, hate, and self-harm content with multiple severity levels.

In [ ]:
# Example - Content Safety (following official quickstart)
from azure.ai.contentsafety import ContentSafetyClient
from azure.ai.contentsafety.models import AnalyzeTextOptions, TextCategory
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError

def analyze_text_content(text_to_analyze):
    """
    Function to analyze text using Azure Content Safety
    Based on Microsoft's official quickstart
    """
    # Get credentials from environment variables
    key = os.getenv('CONTENT_SAFETY_KEY')
    endpoint = os.getenv('CONTENT_SAFETY_ENDPOINT')
    
    if not key or not endpoint:
        print("❌ Error: Define the CONTENT_SAFETY_KEY and CONTENT_SAFETY_ENDPOINT variables in the .env file")
        return None
    
    try:
        # Create Azure AI Content Safety client
        client = ContentSafetyClient(endpoint, AzureKeyCredential(key))
        
        # Configure request
        request = AnalyzeTextOptions(text=text_to_analyze)
        
        # Analyze text
        response = client.analyze_text(request)
        
        # Extract results by specific category (following quickstart)
        hate_result = next((item for item in response.categories_analysis if item.category == TextCategory.HATE), None)
        self_harm_result = next((item for item in response.categories_analysis if item.category == TextCategory.SELF_HARM), None)
        sexual_result = next((item for item in response.categories_analysis if item.category == TextCategory.SEXUAL), None)
        violence_result = next((item for item in response.categories_analysis if item.category == TextCategory.VIOLENCE), None)
        
        # Display results
        print(f"📝 Analyzed text: '{text_to_analyze}'")
        print("🔍 Analysis results:")
        
        if hate_result:
            print(f"  💬 Hate: Severity {hate_result.severity}")
        if self_harm_result:
            print(f"  🩹 Self-harm: Severity {self_harm_result.severity}")
        if sexual_result:
            print(f"  🔞 Sexual: Severity {sexual_result.severity}")
        if violence_result:
            print(f"  ⚔️  Violence: Severity {violence_result.severity}")
        
        # Interpret overall risk level
        max_severity = max([
            hate_result.severity if hate_result else 0,
            self_harm_result.severity if self_harm_result else 0,
            sexual_result.severity if sexual_result else 0,
            violence_result.severity if violence_result else 0
        ])
        
        if max_severity == 0:
            risk_level = "✅ Safe"
        elif max_severity <= 2:
            risk_level = "⚠️ Low risk"
        elif max_severity <= 4:
            risk_level = "🔸 Moderate risk"
        else:
            risk_level = "🔴 High risk"
            
        print(f"📊 Overall assessment: {risk_level}")
        return response
        
    except HttpResponseError as e:
        print("❌ Text analysis failed.")
        if e.error:
            print(f"Error code: {e.error.code}")
            print(f"Error message: {e.error.message}")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None

# Configure Content Safety client
content_safety_endpoint = os.getenv('CONTENT_SAFETY_ENDPOINT')
content_safety_key = os.getenv('CONTENT_SAFETY_KEY')

if content_safety_endpoint and content_safety_key:
    print("=== Content Safety - Text Analysis ===")
    print("Based on Microsoft's official quickstart")
    print("=" * 50)
    
    # Sample texts for analysis (including different risk levels)
    test_texts = [
        "Hello! How are you today? Have a great day!",
        "This is neutral text about technology and Python programming.",
        "I'm very angry about this situation, but I'll resolve it civilly.",
        "Test text for potentially problematic content moderation."
    ]
    
    print(f"📋 Analyzing {len(test_texts)} sample texts...\n")
    
    for idx, text in enumerate(test_texts, 1):
        print(f"🔍 Analysis {idx}/{len(test_texts)}:")
        result = analyze_text_content(text)
        
        if result:
            print("✅ Analysis completed successfully")
        else:
            print("❌ Analysis failed")
            
        print("-" * 50)
        
else:
    print("❌ Required configuration:")
    print("Define the CONTENT_SAFETY_ENDPOINT and CONTENT_SAFETY_KEY variables in the .env file")
    print("\nConfiguration example:")
    print("CONTENT_SAFETY_ENDPOINT=https://your-content-safety.cognitiveservices.azure.com/")
    print("CONTENT_SAFETY_KEY=your-content-safety-key")

In [ ]:
# Example - Content Safety for Images (following official quickstart)
from azure.ai.contentsafety.models import AnalyzeImageOptions, ImageData
from azure.core.exceptions import HttpResponseError

def analyze_image_content(image_path):
    """
    Function to analyze image using Azure Content Safety
    Based on Microsoft's official quickstart
    """
    # Get credentials from environment variables
    key = os.getenv('CONTENT_SAFETY_KEY')
    endpoint = os.getenv('CONTENT_SAFETY_ENDPOINT')
    
    if not key or not endpoint:
        print("❌ Error: Define the CONTENT_SAFETY_KEY and CONTENT_SAFETY_ENDPOINT variables in the .env file")
        return None
    
    try:
        # Create Azure AI Content Safety client
        client = ContentSafetyClient(endpoint, AzureKeyCredential(key))
        
        # Read image
        with open(image_path, "rb") as file:
            image_data = file.read()
            
        # Configure request for image analysis
        request = AnalyzeImageOptions(image=ImageData(content=image_data))
        
        # Analyze image
        response = client.analyze_image(request)
        
        # Extract results by specific category
        hate_result = next((item for item in response.categories_analysis if item.category == TextCategory.HATE), None)
        self_harm_result = next((item for item in response.categories_analysis if item.category == TextCategory.SELF_HARM), None)
        sexual_result = next((item for item in response.categories_analysis if item.category == TextCategory.SEXUAL), None)
        violence_result = next((item for item in response.categories_analysis if item.category == TextCategory.VIOLENCE), None)
        
        # Display results
        print(f"🖼️ Analyzed image: {image_path}")
        print("🔍 Analysis results:")
        
        if hate_result:
            print(f"  💬 Hate: Severity {hate_result.severity}")
        if self_harm_result:
            print(f"  🩹 Self-harm: Severity {self_harm_result.severity}")
        if sexual_result:
            print(f"  🔞 Sexual: Severity {sexual_result.severity}")
        if violence_result:
            print(f"  ⚔️  Violence: Severity {violence_result.severity}")
        
        # Interpret overall risk level
        max_severity = max([
            hate_result.severity if hate_result else 0,
            self_harm_result.severity if self_harm_result else 0,
            sexual_result.severity if sexual_result else 0,
            violence_result.severity if violence_result else 0
        ])
        
        if max_severity == 0:
            risk_level = "✅ Safe"
        elif max_severity <= 2:
            risk_level = "⚠️ Low risk"
        elif max_severity <= 4:
            risk_level = "🔸 Moderate risk"
        else:
            risk_level = "🔴 High risk"
            
        print(f"📊 Overall assessment: {risk_level}")
        return response
        
    except HttpResponseError as e:
        print("❌ Image analysis failed.")
        if e.error:
            print(f"Error code: {e.error.code}")
            print(f"Error message: {e.error.message}")
        return None
    except FileNotFoundError:
        print(f"❌ Error: Image file not found: {image_path}")
        print("Check if the image path is correct.")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None

if content_safety_endpoint and content_safety_key:
    print("\n=== Content Safety - Image Analysis ===")
    print("Based on Microsoft's official quickstart")
    print("=" * 50)
    
    # Paths for sample images
    image_paths = [
        "../../../samples/234039841.jpg",
        "../../../samples/car-accident.png"
    ]
    
    for idx, image_path in enumerate(image_paths, 1):
        print(f"\n🔍 Analysis {idx}/{len(image_paths)}:")
        result = analyze_image_content(image_path)
        
        if result:
            print("✅ Analysis completed successfully")
        else:
            print("❌ Analysis failed")
            
        print("-" * 50)
        
else:
    print("❌ Required configuration:")
    print("Define the CONTENT_SAFETY_ENDPOINT and CONTENT_SAFETY_KEY variables in the .env file")